In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
from  selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time as time
import getpass
import urllib.request
import random
 
 
from time import sleep
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
driver = webdriver.Chrome("/Users/hyerin/ChromeDriver/chromedriver") # Chromedriver PATH

driver.get("https://www.instagram.com/accounts/login/")
driver.maximize_window()

In [3]:
username = getpass.getpass("Input ID : ") # User ID
password = getpass.getpass("Input PWD : ") # User PWD, 특히 getpass를 통해서 비밀번호 정보를 숨길 수도 있다. 잘 배워두자 
hashTag = input("Input HashTag : #") # Search #
 
element_id = driver.find_element_by_name("username")
element_id.send_keys(username)
element_password = driver.find_element_by_name("password")
element_password.send_keys(password)
 
sleep(1.5)

Input ID : ········
Input PWD : ········
Input HashTag : #친환경세제


In [4]:
# login button click
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

In [30]:
driver.get(f'https://www.instagram.com/explore/tags/{hashTag}/')

In [31]:
# 데이터를 저장할 Dictionary
insta_dict = {'id':[],
              'date': [],
              'like': [],
              'text': [],
              'hashtag':[]}
 
 
# 첫 번째 게시물 클릭 
first_post = driver.find_element_by_class_name('eLAPa')
first_post.click()

In [32]:
seq = 0
start = time.time()
 
while True:
    try:
        if driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow'):
            if seq % 20 == 0:
                print('{}번째 수집 중'.format(seq), time.time() - start, sep = '\t')
 
 
 
            # id 정보 수집
            try:
                info_id = driver.find_element_by_css_selector('h2._6lAjh').text
                insta_dict['id'].append(info_id)
            except:
                info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
                insta_dict['id'].append(info_id)
 
 
            # 시간정보 수집 
            time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
            insta_dict['date'].append(time_info)
 
            # like 정보 수집
            try:
                driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5')
                like = driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5').text
                insta_dict['like'].append(like)
 
            except:
                insta_dict['like'].append('영상')
 
 
 
            # text 정보수집
            raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
            text = []
            for i in range(len(raw_info)):
                ## 첫번째 text는 아이디니까 제외 
                if i == 0:
                    pass
                ## 두번째부터 시작 
                else:
                    if '#' in raw_info[i]:
                        pass
                    else:
                        text.append(raw_info[i])
            clean_text = ' '.join(text)
            insta_dict['text'].append(clean_text)
 
            # hashtag 수집
            raw_tags = driver.find_elements_by_css_selector('a.xil3i')
            hash_tag = []
            for i in range(len(raw_tags)):
                if raw_tags[i].text == '':
                    pass
                else:
                    hash_tag.append(raw_tags[i].text)
 
            insta_dict['hashtag'].append(hash_tag)
 
            seq += 1
 
#             if seq == 100:
#                 break
 
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(1.5)
            
 
        else:
            break
            
    except:
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

0번째 수집 중	0.007681131362915039
20번째 수집 중	38.68248438835144
40번째 수집 중	75.9077832698822
60번째 수집 중	112.99396204948425
80번째 수집 중	150.0476269721985


In [34]:
test = pd.DataFrame.from_dict(insta_dict)

In [37]:
test

,id,date,like,text,hashtag
0,the_stamp1,2021-07-08 00:00:00+00:00,영상,코코넛비누 포득포득 넘좋아요🙃 . Reposted from @say.nature _...,"[#코코넛주방비누, #커피주방비누, #친환경세제, #수강생작품, #coconut, ..."
1,__luv_j,2021-05-30 00:00:00+00:00,영상,6월5일 환경의 날을 맞아 일상에서 제로웨이스트를 실천할 수 있는 에코스토어 리필스...,"[#광고, #협찬, #에코스토어, #리필스테이션]"
2,miseo_0526,2021-05-16 00:00:00+00:00,영상,. 요즈음 이라면 이 브랜드 모르는 분들 없으실거에요 바로 뉴질랜드 친환경 세제 브...,"[#공동구매, #홍보, #육아맘, #에코스토어, #미서마켓]"
3,yoonsom._.story,2021-05-27 00:00:00+00:00,영상,"좋아하는 홈웨어와 기분 좋은 향이 만들어준, 설레는 마음으로 보내는 하루 📝 하어스...","[#제품제공, #하어스, #친환경세제, #천연세탁세제]"
4,e_mulgun.today,2021-04-29 00:00:00+00:00,영상,‘즐거운 내일’을 뜻하는 순우리말 🌸 Laundry detergent and fab...,"[#라온하제, #자연을담은, #빨래의향기]"
...,...,...,...,...,...
95,homerunc_s,2021-08-04 00:00:00+00:00,영상,[청소전] 안산시 상록구 부곡동 신축빌라(복층) 친환경 입주청소 디테일과 꼼꼼함이 ...,"[#신축빌라, #빌라복층, #복층, #복층입주청소, #빌라입주청소, #빌라이사청소,..."
96,homerunc_s,2021-08-04 00:00:00+00:00,영상,[청소전] 안산시 상록구 부곡동 신축빌라(복층) 친환경 입주청소 디테일과 꼼꼼함이 ...,"[#신축빌라, #빌라복층, #복층, #복층입주청소, #빌라입주청소, #빌라이사청소,..."
97,homerunc_s,2021-08-04 00:00:00+00:00,영상,도쿄올림픽 야구국가대표 본선라운드 경기일정 시간 (한국:미국) 준결승 . . 홈런크...,"[#4강전, #준결승, #미국, #한국미국, #도쿄올림픽, #도쿄올림픽야구, #야구..."
98,homerunc_s,2021-08-04 00:00:00+00:00,영상,류현진 - 시즌 11승 달성 로저스센터 첫 등판 AL 다승 공동1위 . . 홈런크린...,"[#류현진, #11승, #시즌11승, #다승1위, #로저스센터, #다승공동1위, #..."
